In [1]:
import numpy as np
import igl
import meshplot as mp
from scipy.spatial.transform import Rotation
import ipywidgets as iw
import time
from scipy.sparse import csc_matrix,diags
from scipy.sparse.linalg import spsolve,inv

/home/m/miniconda3/envs/gp/lib/python3.10/site-packages/igl/helpers.py:11: UserWarning: A NumPy version >=1.22.4 and <2.3.0 is required for this version of SciPy (detected version 1.22.3)
  from scipy import sparse


In [2]:
v, f = igl.read_triangle_mesh('data/hand.off')
labels = np.load('data/hand.label.npy').astype(int)
v -= v.min(axis=0)
v /= v.max()

In [3]:
handle_vertex_positions = v.copy()
pos_f_saver = np.zeros((labels.max() + 1, 6))
def pos_f(s,x,y,z, α, β, γ):
    slices = (labels==s)
    r = Rotation.from_euler('xyz', [α, β, γ], degrees=True)
    v_slice = v[slices] + np.array([[x,y,z]])
    center = v_slice.mean(axis=0)
    handle_vertex_positions[slices] = r.apply(v_slice - center) + center
    pos_f_saver[s - 1] = [x,y,z,α,β,γ]
    t0 = time.time()
    v_deformed = pos_f.deformer(handle_vertex_positions)
    p.update_object(vertices = v_deformed)
    t1 = time.time()
    print('FPS', 1/(t1 - t0))
#pos_f.deformer = lambda x:x

In [4]:
def widgets_wrapper():
    segment_widget = iw.Dropdown(options=np.arange(labels.max()) + 1)
    translate_widget = {i:iw.FloatSlider(min=-1, max=1, value=0) 
                        for i in 'xyz'}
    rotate_widget = {a:iw.FloatSlider(min=-90, max=90, value=0, step=1) 
                     for a in 'αβγ'}

    def update_seg(*args):
        (translate_widget['x'].value,translate_widget['y'].value,
        translate_widget['z'].value,
        rotate_widget['α'].value,rotate_widget['β'].value,
        rotate_widget['γ'].value) = pos_f_saver[segment_widget.value]
    segment_widget.observe(update_seg, 'value')
    widgets_dict = dict(s=segment_widget)
    widgets_dict.update(translate_widget)
    widgets_dict.update(rotate_widget)
    return widgets_dict

In [9]:
def position_deformer(target_pos):
    slices = np.where(labels>0)[0]
    mass_mat=igl.massmatrix(target_pos,f)
    M=mass_mat
    cot_mat=csc_matrix(igl.cotmatrix(target_pos,f))
    A=cot_mat@inv(M)@cot_mat
    #b=np.zeros_like(target_pos)
    b = csc_matrix((target_pos.shape[0], target_pos.shape[1]), dtype=np.float64)
    b=-(A[:,slices]@target_pos[slices])
    A=A.tolil()
    b=b.tolil()
    A[slices,:]=0
    A[:,slices]=0
    b[slices,:]=0 #I think it doesnot matter if this is commented
    A=A.tocsc()
    b=b.tocsc()
    x = spsolve(A,b)
    x[slices]=target_pos[slices] #x[slices] should original be 0

    return x

pos_f.deformer = position_deformer


In [10]:
## Widget UI
handle_vertex_positions=position_deformer(handle_vertex_positions)
p = mp.plot(handle_vertex_positions, f, c=labels)
iw.interact(pos_f,
            **widgets_wrapper())

AttributeError: 'numpy.ndarray' object has no attribute 'tolil'